In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import os

import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
from torchvision.transforms import ToTensor, ToPILImage

import wandb

In [2]:
wandb.init(
    project="Final Project", 
    entity="aitech4_cv3",
    name='classification_resnet50',
    config = {
        "lr" : 0.0001,
        "epoch" : 50,
        "batch_size" : 8,
    }
    )

config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jinnyeong (aitech4_cv3). Use `wandb login --relogin` to force relogin


In [3]:
trans = transforms.Compose([
    transforms.Resize((720,1280)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_path = '/opt/ml/classification/input/train'
train_set = torchvision.datasets.ImageFolder(root=train_path, transform=trans)

valid_path = '/opt/ml/classification/input/valid'
valid_set = torchvision.datasets.ImageFolder(root=valid_path, transform=trans)

test_path = '/opt/ml/classification/input/test'
test_set = torchvision.datasets.ImageFolder(root=test_path, transform=trans)

In [5]:
train_set.__getitem__(200)

(tensor([[[-0.1765, -0.1922, -0.1608,  ..., -0.2941, -0.2941, -0.2941],
          [-0.1294, -0.1451, -0.1529,  ..., -0.2941, -0.2941, -0.2941],
          [-0.1059, -0.1216, -0.1373,  ..., -0.2941, -0.2941, -0.2941],
          ...,
          [-0.7569, -0.7647, -0.7647,  ..., -0.7804, -0.7725, -0.7882],
          [-0.7725, -0.7725, -0.7647,  ..., -0.7490, -0.7490, -0.7647],
          [-0.7725, -0.7725, -0.7725,  ..., -0.7569, -0.7725, -0.7882]],
 
         [[-0.1922, -0.2000, -0.1686,  ..., -0.7882, -0.7882, -0.7882],
          [-0.1137, -0.1216, -0.1373,  ..., -0.7882, -0.7882, -0.7882],
          [-0.0431, -0.0588, -0.0824,  ..., -0.7882, -0.7882, -0.7882],
          ...,
          [-0.6706, -0.6784, -0.6784,  ..., -0.7176, -0.7098, -0.7176],
          [-0.6784, -0.6784, -0.6784,  ..., -0.6863, -0.6784, -0.6941],
          [-0.6784, -0.6784, -0.6784,  ..., -0.6863, -0.7020, -0.7176]],
 
         [[-0.3333, -0.3412, -0.3176,  ..., -0.8588, -0.8588, -0.8588],
          [-0.2157, -0.2314,

In [6]:
train_set.classes

['0', '1']

In [7]:
train_loader = DataLoader(
    train_set,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=4
)
valid_loader = DataLoader(
    valid_set,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=4
)
test_loader = DataLoader(
    test_set,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=4
)

In [8]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.shape, images.type())
print(labels.shape, labels.type())
print(labels)

torch.Size([8, 3, 720, 1280]) torch.FloatTensor
torch.Size([8]) torch.LongTensor
tensor([0, 1, 0, 1, 0, 1, 1, 0])


In [9]:
class MyModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        self.backbone = models.resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [12]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch=config.epoch):
    train_losses = []
    train_correct = 0
    log_interval = 10

    for batch_idx, (inputs, label) in enumerate(train_loader):

        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        
        inputs = inputs.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        # batch_size = inputs.size()[0]
        # inputs = inputs.view(-1,64) # model의 input size에 맞춰야 함
        
        outputs = network(inputs)

        loss = loss_func(outputs,label)
        train_losses.append(loss.item())

        pred = torch.max(outputs,1)[1] # max index 추출 // torch.argmax도 가능
        train_correct += pred.eq(label).sum() # 맞춘 갯수 추출

        loss.backward()

        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
            .format(epoch, batch_idx*len(label), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
            loss.item()))

        

    return train_losses, train_correct

In [13]:
def valid_epoch(valid_loader, network, loss_func):
    correct = 0
    valid_losses = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(valid_loader):

            use_cuda = torch.cuda.is_available()
            device = torch.device("cuda" if use_cuda else "cpu")

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            batch_size = inputs.size()[0]
            # inputs = inputs.view(-1,64)

            outputs = network(inputs)

            loss = loss_func(outputs, labels)
            valid_losses.append(loss.item())

            pred = torch.max(outputs,1)[1]
            correct += pred.eq(labels).sum()

        valid_accuracy = 100. * correct / len(valid_loader.dataset)

        print('Valid set: Accuracy: {}/{} ({:.0f}%)\n'
              .format(correct, len(valid_loader.dataset),100. * correct / len(valid_loader.dataset)))

        return valid_losses, valid_accuracy

In [14]:
def training(network, learning_rate = config.lr):

      best_accuracy = 0.0
      best_epoch = 0
      epoch = 50

      cls_loss = nn.CrossEntropyLoss()
      optimizer = optim.Adam(network.parameters(), lr=learning_rate)

      train_losses_per_epoch = []
      valid_losses_per_epoch = []

      train_accuracies = []
      valid_accuracies = []

      for epoches in range(epoch):

            network.train()

            train_losses, train_correct = training_epoch(train_loader, network, cls_loss, optimizer, epoches)

            average_loss = np.mean(train_losses)
            train_losses_per_epoch.append(average_loss)

            train_accuracy = 100. * train_correct / len(train_loader.dataset)
            train_accuracies.append(train_accuracy)

            print('\nTraining set: Accuracy: {}/{} ({:.0f}%)'
                  .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

            network.eval()

            correct = 0
            
            with torch.no_grad():
                  valid_losses, valid_accuracy = valid_epoch(valid_loader, network, cls_loss)
            
                  valid_losses_per_epoch.append(np.mean(valid_losses))
                  valid_accuracies.append(valid_accuracy)

                  if valid_accuracy > best_accuracy:
                        best_accuracy = valid_accuracy

                        if epoches != 0:
                              before_path = '/opt/ml/classification/pth/best_epoch{}.pth'.format(best_epoch)
                              os.remove(before_path)

                        best_epoch = epoches

                        path = '/opt/ml/classification/pth/best_epoch{}.pth'.format(epoches)
                        torch.save(network.state_dict(), path)

                        print('save best.pth')

            if epoches == epoch:
                  path = '/opt/ml/classification/pth/latest.pth'
                  torch.save(network.state_dict(), path)
                  print('save latest.pth')
                  print('done')

            wandb.log({'train_acc':train_accuracy, 'val_acc':valid_accuracy})
      
      return train_losses_per_epoch, valid_losses_per_epoch, train_accuracies, valid_accuracies


In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

network = MyModel().to(device)
model_res50 = training(network)

Train Epoch: 0 [0/800 (0%)]	Loss: 1.014529
Train Epoch: 0 [80/800 (10%)]	Loss: 0.666609
Train Epoch: 0 [160/800 (20%)]	Loss: 0.272643
Train Epoch: 0 [240/800 (30%)]	Loss: 0.467650
Train Epoch: 0 [320/800 (40%)]	Loss: 0.228057
Train Epoch: 0 [400/800 (50%)]	Loss: 0.284327
Train Epoch: 0 [480/800 (60%)]	Loss: 0.469987
Train Epoch: 0 [560/800 (70%)]	Loss: 0.316821
Train Epoch: 0 [640/800 (80%)]	Loss: 0.668304
Train Epoch: 0 [720/800 (90%)]	Loss: 0.767242

Training set: Accuracy: 610/800 (76%)
Valid set: Accuracy: 85/100 (85%)

save best.pth
Train Epoch: 1 [0/800 (0%)]	Loss: 0.332150
Train Epoch: 1 [80/800 (10%)]	Loss: 0.209632
Train Epoch: 1 [160/800 (20%)]	Loss: 0.200642
Train Epoch: 1 [240/800 (30%)]	Loss: 0.204539
Train Epoch: 1 [320/800 (40%)]	Loss: 1.136284
Train Epoch: 1 [400/800 (50%)]	Loss: 0.827948
Train Epoch: 1 [480/800 (60%)]	Loss: 0.110589
Train Epoch: 1 [560/800 (70%)]	Loss: 0.119816
Train Epoch: 1 [640/800 (80%)]	Loss: 0.225209
Train Epoch: 1 [720/800 (90%)]	Loss: 0.452507


In [24]:
def test(network):
    best_path = '/opt/ml/classification/pth/best_epoch31.pth' # test마다 변경

    network.load_state_dict(torch.load(best_path))

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            
            use_cuda = torch.cuda.is_available()
            device = torch.device("cuda" if use_cuda else "cpu")

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = network(inputs)

            pred = torch.max(outputs,1)[1]

            print('Prediction')
            print(pred)
            print('Label')
            print(labels)
            print()

            # if pred == 1:
            #     print('A pothole is detected')
            # else:
            #     print('A pothole is not detected')

In [25]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

test_model = MyModel().to(device)
resetnet50_test = test(test_model)


Prediction
tensor([1, 0, 0, 0, 1, 1, 1, 1], device='cuda:0')
Label
tensor([1, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')

Prediction
tensor([1, 1, 0, 0, 1, 0, 1, 0], device='cuda:0')
Label
tensor([1, 1, 0, 0, 1, 0, 0, 0], device='cuda:0')

Prediction
tensor([1, 0, 0, 0, 0, 1, 1, 1], device='cuda:0')
Label
tensor([1, 0, 0, 1, 1, 1, 1, 1], device='cuda:0')

Prediction
tensor([1, 1, 0, 0, 1, 0, 1, 0], device='cuda:0')
Label
tensor([1, 0, 0, 0, 1, 0, 1, 1], device='cuda:0')

Prediction
tensor([0, 1, 1, 0, 0, 1, 1, 0], device='cuda:0')
Label
tensor([1, 1, 1, 0, 1, 1, 1, 0], device='cuda:0')

Prediction
tensor([0, 1, 1, 1, 0, 1, 1, 0], device='cuda:0')
Label
tensor([0, 1, 1, 0, 0, 0, 1, 0], device='cuda:0')

Prediction
tensor([0, 1, 1, 1, 1, 0, 0, 0], device='cuda:0')
Label
tensor([0, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')

Prediction
tensor([1, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')
Label
tensor([1, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')

Prediction
tensor([1, 1, 1, 0, 0, 1, 0, 0], device='cuda